# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
from bs4 import BeautifulSoup

In [5]:
# список интересных ключевых слов для поиска
# KEYWORDS = ['python', 'парсинг']
keywords = ['систем', 'информац']
url = 'https://habr.com/ru/all/'

In [6]:
def news_list_request(url, keywords):
    '''
    Функция с сайта Хабр выгружает список блоков html-кода со свежими статьями, содержащими запрашиваемые ключевые слова
    '''
    news_list = []

    # подготавливаем данные с сайта: записываем страницу в переменную habr, переводим её в объект
    # BeatifulSoup переменную habr_soup, получаем список всех вхождений тега div с классом tm-article-snippet
    # (так помечены блоки со статьями)

    habr = requests.get(url)
    habr_soup = BeautifulSoup(habr.text, 'html.parser')
    news = habr_soup.find_all('div', class_='tm-article-snippet')

    # создаём список новостей news_list, в которые входит хотя бы одно из ключевых слов из переменной keywords
    for item in news:
        for word in keywords:
            if word in item.text:
                news_list.append(item)
                break
    return news_list

In [7]:
news_list = news_list_request(url, keywords)

In [38]:
def new_articles_df(news_list):
    '''
    Функция создаёт Dataframe new_articles из списка новостей news_list
    '''
    new_articles = pd.DataFrame()
    for news in news_list:
        date = pd.to_datetime(news.find('time').get('datetime')).date() #вынимаем дату
        header = news.find('a', class_='tm-article-snippet__title-link').text #вынимаем заголовок
        link = 'https://habr.com' + news.find('a', class_='tm-article-snippet__title-link').get('href') #вынимаем ссылку
        row = {'date': date, 'header': header, 'link': link} #создаём строку для последующего присоединения к new_articles
        new_articles = pd.concat([new_articles,pd.DataFrame([row])])
    new_articles.reset_index()
    return new_articles

In [41]:
new_articles = new_articles_df(news_list)
new_articles

,date,header,link
0,2021-09-07,Создаем текстовый редактор на React.js,https://habr.com/ru/company/kts/blog/576682/
0,2021-09-07,"Android in-app purchases, часть 5: серверная в...",https://habr.com/ru/company/adapty/blog/576164/
0,2021-09-07,"ClickHouse: путь джедая, искавшего дом для сво...",https://habr.com/ru/company/tuturu/blog/576632/
0,2021-09-07,"Чем рука робота лучше вашей, или Автоматизация...",https://habr.com/ru/company/lanit/blog/575714/


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [98]:
EMAIL =['abc@yandex.ru', 'def@yandex.ru']

avast_request_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

params = {
    'emailAddresses': EMAIL,
}

Headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '39',
    'Content-Type': 'application/json;charset=UTF-8',
    'DNT': '1',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 YaBrowser/21.8.1.468 Yowser/2.5 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

def hack_leaks(request_url, Headers, params):
    res_dict = requests.post(request_url, headers=Headers, json=params).json()

    leaks = pd.DataFrame()
    for breach in res_dict['breaches']:
        date = res_dict['breaches'][breach]['publishDate']
        source = res_dict['breaches'][breach]['site']
        description = res_dict['breaches'][breach]['description']
        row = {
            'date': date,
            'source': source,
            'description': description
        }
        leaks = pd.concat([leaks, pd.DataFrame([row])])
    leaks = leaks.reset_index()[['date', 'source', 'description']]
    return leaks
    
hack_leaks(avast_request_url, Headers, params)

,date,source,description
0,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
1,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
2,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
3,2017-04-14T00:00:00Z,evgexacraft.ru,"At an unconfirmed date, EvgexaCraft's user dat..."
4,2019-05-23T00:00:00Z,livejournal.com,"In 2017, social network LiveJournal's database..."
5,2019-09-12T00:00:00Z,xrumer.winguild.ru,"At an unconfirmed date, Russian audio equipmen..."
6,2020-02-20T00:00:00Z,shop.westwing.ru,"In November 2019, the Russian interior design ..."
7,2020-04-09T00:00:00Z,vkmix.com,"At an unconfirmed date, the Russian social med..."
8,2020-09-17T00:00:00Z,eng.arendator.ru,"At an unconfirmed date, the Russian commercial..."
9,2021-02-04T00:00:00Z,rzd-bonus.ru,"In June 2020, the Russian Railways Bonus Progr..."


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
